## Roadsigns Data Collection

# Installing Selenium

In [30]:
pip install selenium

### Starting the web driver

In [52]:
import selenium
import requests
import io
from selenium import webdriver
import hashlib
# Put the path for your ChromeDriver here
DRIVER_PATH = 'D:\Vedanth\proxvision\chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [53]:
wd.get('https://google.com')


When you run the above two cells a window with google.com should open

Search for Dogs

In [54]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

In [55]:
wd.quit()
#Close the driver

The function fetch_image_urls expects three input parameters:<br>
query : Search term, like Dog<br>
max_links_to_fetch : Number of links the scraper is supposed to collect<br>
webdriver : instantiated Webdriver

In [56]:
import time
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


### Downloading images with Pillow

In [57]:
pip install Pillow

In [58]:
from PIL import Image


The persist_image function grabs an image URL url and downloads it into the folder_path. The function will assign the image a random 10-digit id.


In [59]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


Now let's combine above two functions and make a single short and concise function

In [60]:
import os
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

## Entering out search terms and downloading the images

In [61]:
search_term = "dogs"
search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH
    )

Found: 49 search results. Extracting links from 0:49
Found: 5 image links, done!
SUCCESS - saved https://www.nationalgeographic.com/content/dam/animals/thumbs/rights-exempt/mammals/d/domestic-dog_thumb.ngsversion.1546554600360.adapt.1900.1.jpg - as ./images\dogs\a966bef2ff.jpg
SUCCESS - saved https://i.insider.com/5484d9d1eab8ea3017b17e29?width=600&format=jpeg&auto=webp - as ./images\dogs\b50bad13c7.jpg
SUCCESS - saved https://www.cdc.gov/healthypets/images/pets/cute-dog-headshot.jpg - as ./images\dogs\af7ec217c8.jpg
SUCCESS - saved https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/article_thumbnails/other/dog_cool_summer_slideshow/1800x1200_dog_cool_summer_other.jpg - as ./images\dogs\37f0960b6e.jpg
SUCCESS - saved https://media.nature.com/lw800/magazine-assets/d41586-020-01430-5/d41586-020-01430-5_17977552.jpg - as ./images\dogs\36c2df0788.jpg
